In [634]:
import dendropy
from dendropy.simulate import treesim
from dendropy.calculate import treecompare
import random

In [635]:
def print_generation(n):
    familly['F{}L_filtered'.format(n)].print_plot()
    print(familly['F{}L_mutation'.format(n)])

    familly['F{}R_filtered'.format(n)].print_plot()
    print(familly['F{}R_mutation'.format(n)])
    print(familly['F{}_URF'.format(n)])

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def swaaap(fam):
    swapy = []
    for n in fam.postorder_internal_node_iter():
    
        if (random.random() > 0.9 and n._parent_node and n._child_nodes ):
            swapy.append(n)
            
    if len(swapy) >= 2 :
    
        if len(swapy)%2 == 1:
            swapy.pop()
                
    
        for pair in chunks(swapy, 2):
            

            s0 = pair[0].extract_subtree()
            p0 = pair[0]._parent_node

            s1 = pair[1].extract_subtree()
            p1 = pair[1]._parent_node

            p0.add_child(s1)
            p1.add_child(s0)
            
            p0.remove_child(pair[0])
            p1.remove_child(pair[1])
            
    
    for n in fam.leaf_node_iter():
        if (n.taxon == None):
            n._parent_node.remove_child(n)
            

In [636]:
forest = []
size_forest = 200
n_generations = 4

for e in range(size_forest):
    
    to_store = {}
    
        
    familly = {}
    familly['tree_size'] = e+4
    to_store['tree_size'] = familly['tree_size']
    familly['tns'] = dendropy.TaxonNamespace(['T_{}'.format(i) for i in range(1,int(e * 1.5))])

    # ROOT PAIR
    familly['ROOT'] = treesim.birth_death_tree(1, 0.1, taxon_namespace=familly['tns'], num_extant_tips=familly['tree_size'])
    
    # RANDOM FILTER LEAF
    familly['L'] = dendropy.Tree(familly['ROOT'])
    familly['L'].filter_leaf_nodes(lambda n: random.random() < 0.8, recursive=False, update_bipartitions=True, suppress_unifurcations=True)
    
    familly['R'] = dendropy.Tree(familly['ROOT'])
    familly['R'].filter_leaf_nodes(lambda n: random.random() < 0.8, recursive=False, update_bipartitions=True, suppress_unifurcations=True)
   
    # RANDOM SWAP NODE
    if familly['tree_size'] > 30:
        swaaap(familly['R'])
        swaaap(familly['L'])
    
    to_store['L'] = familly['L'].as_string(schema='newick')
    to_store['R'] = familly['R'].as_string(schema='newick')
    
    familly['common'] = set(n.taxon for n in familly['L'].leaf_node_iter()) & set(n.taxon for n in familly['R'].leaf_node_iter())
    familly['common'].discard(None)
    
    familly['L_filter'] = familly['L'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    familly['R_filter'] = familly['R'].extract_tree(node_filter_fn=lambda n: n.taxon in familly['common'])
    
    # CLADE
    familly['R_filter'].is_rooted = True
    familly['L_filter'].is_rooted = True
    to_store['root_clade'] = treecompare.symmetric_difference( familly['L_filter'], familly['R_filter'])

    
    to_store['L_filter'] = familly['L_filter'].as_string(schema='newick')
    to_store['R_filter'] = familly['R_filter'].as_string(schema='newick')
    
    
   
    # RF + EUC
    familly['R_filter'].is_rooted = False
    familly['L_filter'].is_rooted = False
        
    to_store['root_URF'] = treecompare.symmetric_difference( familly['L_filter'], familly['R_filter'])
    to_store['root_WRF'] = treecompare.weighted_robinson_foulds_distance(familly['L_filter'], familly['R_filter'])
            
    

    forest.append(to_store)
        



In [637]:
import json

with open('distance_testing.json', 'w') as fp:
    json.dump(forest, fp)

In [633]:
n = 35
t1 = dendropy.Tree.get_from_string(forest[n]['L'], schema="newick", rooting="default-rooted")


print([ e.taxon for e in list(t1.leaf_node_iter()) ] )

print(forest[n]['L'])
print(forest[n]['R'])



print(forest[n]['L_filter'])







print(forest[n]['R_filter'])





[<Taxon 0x143c9bdc0 'T_4'>, <Taxon 0x11694fd30 'T_47'>, <Taxon 0x141e7c2e0 'T_36'>, <Taxon 0x116623b20 'T_6'>, <Taxon 0x115e6b130 'T_45'>, <Taxon 0x140d24520 'T_50'>, <Taxon 0x1444a7070 'T_17'>, <Taxon 0x1444a7250 'T_44'>, <Taxon 0x1444a7310 'T_26'>, <Taxon 0x1444a75b0 'T_13'>, <Taxon 0x1444a7670 'T_34'>, <Taxon 0x1444a7730 'T_37'>, <Taxon 0x1444a7850 'T_39'>, <Taxon 0x1444a7970 'T_21'>, <Taxon 0x1444a7a90 'T_46'>, <Taxon 0x1444a7b50 'T_28'>, <Taxon 0x1444a7c10 'T_16'>, <Taxon 0x1444a7d90 'T_27'>, <Taxon 0x1444a7e50 'T_11'>, <Taxon 0x1444a7f10 'T_35'>, <Taxon 0x1444821c0 'T_2'>, <Taxon 0x144482940 'T_48'>]
[&R] ((('T_4':0.06492961326792783,'T_47':0.06492961326792783):0.9614829204370432,(('T_36':0.16250371980926595,('T_6':0.06661375897490657,'T_45':0.06661375897490657):0.09588996083435938):0.6863426979933948,('T_50':0.5670205301446998,'T_17':0.5670205301446998):0.28182588765796085):1.0576218938454944):1.384148500889453,((('T_44':0.8988888623172768,'T_26':0.8988888623172768):0.7713429257